# MVPA 1: Pattern Classification

This tutorial runs a simple pattern classification (decoding) analysis on the localizer data. We will be using many of the tools we've used so far, and also [scikit-learn](https://scikit-learn.org/stable/index.html) for pattern classification. Scikit-learn is an amazing machine learning package that is widely used. We will import some classifiers as well as ways to train these algorithms on our data.

In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, LeaveOneGroupOut

import nibabel as nib
from nilearn.maskers import MultiNiftiMasker, NiftiMasker
from nilearn import image
from nilearn.plotting import plot_roi

%matplotlib inline
pd.options.mode.chained_assignment = None 

## 1. Setting up our ROI mask

Like in many decoding analyses, we'll look at patterns of voxels from a predetermined ROI (there are exceptions to this, see Searchlight). 

For examples sake, we'll use somatosensory cortex, SSc. Remember last time we showed that SSc had similar mean activity for Arm, Grasp, and Touch. To show this, we tossed away a lot of information by averaging across voxels. Pattern classification let's us ask a more refined question: are the _patterns_ of voxel activity in SSc different between these conditions, despite their similarities in mean? Here, we are making use of all activity in the ROI. 

First, let's load in our mask. 

In [ ]:
roi = nib.load('L_SSc.nii.gz')
roi.shape

This mask is in MNI space (as is our data), but is in 2mm isovoxel. Our data, meanwhile, is in 3x3x4 MNI space. Let's load in our functional images and check:

In [ ]:
# get all motor task runs of one subject
glob_template = ('data/derivatives/fmriprep/sub-03/func/'
                '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz')
func_data = glob.glob(glob_template)

# reorder these based on run number 
func_imgs = sorted(list(func_data))
func_imgs

In [ ]:
nib.load(func_imgs[0]).shape

Let's resample our mask so that it has the same number of voxels as our functional data. We are doing this so that we analyze data in according to the functional resolution rather than the mask resolution. This means fewer, and less redundant, voxels (no upsampling to mask resolution) for our pattern classifier to use. 

In [ ]:
# `nearest` ensures that voxels are either 0 or 1 after interpolation
roi = image.resample_to_img('L_SSc.nii.gz', func_imgs[0], interpolation='nearest')
plot_roi(roi)

## 2. Extracting voxel data from our ROI

Now we need to extract voxel data from our ROI. We can use `MultiNiftiMasker` to do this, which lets us pass in a list of functional (unlike `NiftiMasker`). 

Note here that we'll apply some temporal filtering on the data, and regress out residual head motion effects in the data. This is unlike with our first level analysis, where we applied this step as regressors in our model. We'll also standardize the voxels.

Let's get those head motion regressors first:

In [7]:
glob_template = ('data/derivatives/fmriprep/sub-03/func/'
                '*task-motor*confounds_regressors.tsv')
conf_files = glob.glob(glob_template)

# reorder these based on run number
conf_files = sorted(list(conf_files))

# load only the 6 motion parameters into python
motion_params = []
for conf in conf_files:
    conf_data = pd.read_table(conf, usecols=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'])
    motion_params.append(conf_data.values)


Now we can extract our data:

In [ ]:
masker = MultiNiftiMasker(
    mask_img=roi, high_pass=.01, detrend=True, t_r=2, standardize=True
)

func_data = masker.fit_transform(func_imgs, confounds=motion_params)

Now let's see what we have using matplotlib's `imshow`. This will let us plot the ROI's voxels during one functional run. 

In [ ]:
fig, ax = plt.subplots(figsize=(3, 12))
ax.imshow(func_data[0])
ax.set(xlabel='Voxels', ylabel='Trials');

## 3. Getting trial patterns 

Now we need to only **one pattern of voxels per trial**. There are lots of ways of doing this, and several papers that explore how to apply GLM's in order to estimate single trial patterns (e.g., see the excellent work by Mumford et al: [1](https://www.sciencedirect.com/science/article/abs/pii/S1053811911010081), [2](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0126255)). 

For the sake of an excercise/tutorial, we will instead take a pattern roughly corresponding to the peak BOLD activity in a 20s block. This is not an _ideal_ way because we're selecting one timepoint of voxels (so there will be lots of noise), but nevertheless a simple way. 

So, we have to a) read in the event files, b) select a volume after block/trial onset, and c) use that volume number as a row index for the voxel array above. 

Let's get our event files first:

In [ ]:
event_files = glob.glob('data/task-motor*events.tsv')
event_files = sorted(list(event_files))

events = []
for i, ev in enumerate(event_files):
    events.append(pd.read_table(ev))

# show the first 10 rows
events[0].head(10)

Now let's create a function that gives us the volume (row) we wish to use for pattern classification. 

In [11]:
def get_trial_volumes(df, shift=5):
    
    df = df.copy()
    # convert seconds to volume
    df['onset_vols'] = df['onset'] / 2
    
    # get volume corresponding to onset + shift. Default is
    # 5 to approximate peak reponse for a 20s block. We essentially
    # just want something representative of the entire block
    df['trial_vol'] = df['onset_vols'] + shift
    
    # convert to integers rather than floats
    df['trial_vol'] = df['trial_vol'].astype(int)
    
    # returns a dataframe with 3 columns
    return df[['trial', 'trial_type', 'trial_vol']]

Let's use one event file to see what our function does:

In [ ]:
first_run_trials = get_trial_volumes(events[0])
first_run_trials

But typically, we only want two conditions of interests for our classifier in MVPA (i.e. binary classification). So we actually need to pull out just the patterns belonging to these conditions. 

In [ ]:
arm_vs_touch = first_run_trials.query("trial_type == 'Arm' or trial_type == 'Touch'")
arm_vs_touch

Selecting those volumes and plotting the voxels:

In [ ]:
first_run = func_data[0]
trial_voxels = first_run[arm_vs_touch['trial_vol'], :]

fig, ax = plt.subplots()
ax.imshow(trial_voxels)
ax.set(xlabel='Voxels', ylabel='Trials');

Now, I've created a handy function that does this across runs. It iterates through each run, and gives you a final voxel array and row labels.

In [15]:
def get_trial_voxels(voxels, task_events, conditions):
    # iterate through data and event_files in tandem
    
    cond1, cond2 = conditions
    
    trials_list = []
    data_list = []
    for i, (vox, ev) in enumerate(zip(voxels, task_events)):
        
        # get the trial patterns for specific conditions we want to use
        trials = get_trial_volumes(ev)
        trials = trials.query("trial_type == @cond1 or trial_type == @cond2")
        data = vox[trials.loc[:, 'trial_vol'], :]
        
        # get condition labels from trial type column
        trials.loc[:, 'run'] = i
        
        trials_list.append(trials)
        data_list.append(data)
    
    # concatenate lists to create single dataframe and voxel array
    trials = pd.concat(trials_list)
    data = np.vstack(data_list)
        
    return trials, data

We can run it, and it returns information on the trial (trial type, number, etc) and the voxels:

In [ ]:
info, voxels = get_trial_voxels(func_data, events, ['Arm', 'Touch'])
info

In [ ]:
fig, ax = plt.subplots()
ax.imshow(voxels)
ax.set(xlabel='Voxels', ylabel='Trials');

## 4. Running a classifier

Now we can use [scikit learn](https://scikit-learn.org/stable/index.html) to train and evaluate our model. First, we need to define our classifier. At the top, we imported `SVC`, which is support vector machine (SVM) classifier, and `LogisticRegression`, which is a regularized logistic regression classifier. These are probably the two most popular algorithms in decoding studies as the tend to perform well (SVM generally moreso than logistic regression). 


**SVMs:**  
A SVM essentially tries to find a hyperplane between classes with maximal margins. The margins are defined by the closest points to the hyperplane. This is easy to visualize in with two features (e.g., two voxels):

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_separating_hyperplane_0011.png)

The figure above is taken from the excellent scikit-learn documentation: https://scikit-learn.org/stable/modules/svm.html#svm-classification.

In our case, were trying to find a hyperplane in _N_-dimensional space, where _N_ = number of voxels. SVMs perform very well on the data, as they're fairly robust to noise and redundancy.

**Logistic Regression:**  
Scikit learn implements a regularized ($\ell_2$-norm) logistic regression by default (which differs from your standard logistic regression model that has no regularization). Logistic regression essentially transforms the output of a linear regression using a sigmoid function, which produces probabilities between 0 and 1. Binary classification is just done by taking probability values below or above 50% and assigning them as 0 or 1, respectively. 


![](https://scikit-learn.org/stable/_images/sphx_glr_plot_logistic_001.png)

The graph above is created by this example: https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic.html#sphx-glr-auto-examples-linear-model-plot-logistic-py

See also: https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

In [18]:
svm = SVC(kernel='linear')
logreg = LogisticRegression()

**Cross Validation**

Next, we need to set up a way to train our model and then evaluate it. In supervised machine learning, models are first fit on training data. This process essentially lets the model map input data to labels/outcomes. Then the trained models are tested on unseen data to try and predict their labels/outcomes. In our case, we will train our model on voxel data from two conditions, and then use a held-out, unseen test set to try and predict the condition of those data based on their voxels.  

We can do this using [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). Cross validation is an iterative process in which we split our data into train and test folds. Each iteration involves a unique split, thus letting us train and test across all of our data. For each iteration, we obtain a classification accuracy, and we can compute an average accuracy across iterations. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/1280px-K-fold_cross_validation_EN.svg.png)

Scikit-learn makes this process increadibly simple using `cross_val_score`. This does everything for us. All we need to do is give it our data, our model, and how we want to cross validate and score the procedure (e.g., classification accuracy).

So, let's make sure we're clear on our data. Lets set up `X` and `y`, which is typical notation you'll see in scikit-learn docs and elsewhere. 

In [19]:
info, X = get_trial_voxels(func_data, events, ['Arm', 'Touch'])

In [ ]:
y = info['trial_type'].tolist()
y

Now let's set up our cross validator. How do we determine our splits? People typically use _K_-fold cross validation shown above, but in neuroimaging, we often use _leave-one-run-out_ cross validation. This essentially lets our functional runs determine our folds, and ensures that our train/test folds are completely independent from one another (which reduces overfitting by reducing shared variance between train and test sets). 

Let's run `cross_val_score` and give it our model and the data. Note that `groups` is used to define our functional runs (i.e. folds). 

In [ ]:
accuracies = cross_val_score(svm, X, y, groups=info['run'], cv=LeaveOneGroupOut())
accuracies

In [ ]:
np.mean(accuracies)

## 5. Group Analysis

We can put everything we've done into one big for-loop that iterates across subjects. For each subject, we'll run the pattern classification pipeline above and take their mean accuracy. Then we can show these accuracies at the group level.

In [ ]:
# set up parameters
subject_numbers = ['01', '02', '03', '04', '05', '06', '07', '08']
columns = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
roi = 'L_SSc.nii.gz'
conditions = ['Arm', 'Touch']
clf = SVC(kernel='linear')

# resample our mask
roi = image.resample_to_img(roi, func_imgs[0], interpolation='nearest')

results = []
for subject in subject_numbers:
    
    ## 1. Get the functional images in order
    glob_template = (f'data/derivatives/fmriprep/sub-{subject}/func/'
                     '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz')
    func_imgs = glob.glob(glob_template)
    func_imgs = sorted(list(func_imgs))
    
    # 2. Get the regressors
    glob_template = (f'data/derivatives/fmriprep/sub-{subject}/func/'
                      '*task-motor*confounds_regressors.tsv')
    conf_files = glob.glob(glob_template)
    # put files in order
    conf_files = sorted(list(conf_files))

    # load only the 6 motion parameters
    motion_params = []
    for conf in conf_files:
        conf_data = pd.read_table(conf, usecols=columns)
        motion_params.append(conf_data.values)
        
    # extract data
    masker = MultiNiftiMasker(mask_img=roi, high_pass=.01, detrend=True, t_r=2, 
                              standardize=True)
    func_data = masker.fit_transform(func_imgs, confounds=motion_params)
    
    # get trial patterns with trial information
    info, voxels = get_trial_voxels(func_data, events, conditions)
    
    # run cross validation
    X = voxels
    y = info['trial_type'].tolist()
    accuracies = cross_val_score(clf, X, y, groups=info['run'], cv=LeaveOneGroupOut())
    
    # save subject mean accuracy
    results.append(np.mean(accuracies))
    


In [ ]:
# put into dataframe that's convenient for seaborn
df = pd.DataFrame({'x': '', 'accuracy': results})

fig, ax = plt.subplots(figsize=(3, 6))
sns.barplot(x='x', y='accuracy', data=df, ci=68, ax=ax, color='C7')
sns.stripplot(x='x', y='accuracy', data=df, ax=ax, color='k',
              edgecolor='k', marker='o', size=4)
ax.set(ylim=(.4, 1), xlabel='')
ax.axhline(.5, c='k', ls='--')

We can see that the classification accuracy is above chance value of 50%! It's not amazing, but this would pass in a publication. Remember, we're taking _very_ noisy and crude signals of neural activity and asking if we can see any meaningful patterns in them. The fact that we can in the first place is impressive. 

If this lecture was a lot, that's okay. This is a heavy-duty topic that is difficult to grasp in a quick tutorial. The big take away is to know that this approach exists, and to get a quick glimpse of what goes into it when you read papers or decide on analyses in your own projects.  